##Qwen2를 이용한 RAG를 위한 LLM파인튜닝
앞서 정리했던 RAG 학습 데이터셋을 이용하여 LLM 파인튜닝을 진행해보겠습니다. 이 실습은 런팟(Runpod) 클라우드에서 A100 SXM GPU 를 사용하여 진행하였습니다.

###1.패키지 설치 및 데이터로드
이제 필요한 라이브러리를 설치하고 데이터를 다운로드 해보겠습니다.

In [ ]:
!pip install torch==2.4.0 transformers==4.45.1 datasets==3.0.1 accelerate==0.34.2 trl==0.11.1 peft==0.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

각 도구들의 쓰임새는 다음과 같습니다.
- load_dataset, Dataset: 다양한 데이터셋을 쉽게 로드하고 처리할 수 있습니다. load_dataset 은 허깅페이스에 업로드 된 데이터셋을 불러오거나 로컬 파일을 로드하는 데 사용되며, Dataset 은 개별 데이터셋 객체를 다룰 때 사용됩니다.
- AutoModelForCausalLM: 허깅페이스 Transformers 라이브러리에서 제공하는 모델 다운로드를 위한 도구입니다. 언어 모델을 로드하는 데 사용됩니다.
- AutoTokenizer: 특정 모델에 맞는 토크나이저를 자동으로 불러오는 도구입니다. 토크나이저는 텍스트를 언어 모델이 처리할 수 있는 정수 시퀀스로 변환하거나 정수 시퀀스를 다시 텍스트 문자열로 복원하는 역할을 합니다.
- LoraConfig: 이번 실습에서 사용할 학습 방법인 LoRA(Low‑Rank Adaptation) 학습 방식을 사용할때 필요한 설정값을 정의합니다. 학습 시에 거대 언어 모델 전체를 업데이트하는 것이 아니라, 거대
언어 모델의 특정 부분만 업데이트하여 보다 효율적으로 학습하는 방식입니다. 여기서는 로라 튜닝을 할 때 로라 튜닝과 연관된 각종 설정값들을 결정합니다.
- SFTConfig: 모델을 학습할 때 필요한 다양한 설정값을 정의하는 도구입니다. 학습 과정에서 모델이 어떻게 업데이트될지를 조정하며 여기에는 학습률, 배치 크기, 옵티마이저 등의 설정을 포함합
니다. 모델 전체를 학습할 때도 쓰일 수 있지만, 현재 실습과 같이 로라 튜닝에서처럼 모델의 특정부분만 학습하는 방식에서도 사용합니다. SFTConfig 에서 설정하는 이 설정값들은 모델의 학습성능과 안정성에 큰 영향을 줍니다. 예를 들어, 학습률이 너무 크면 학습이 불안정하고, 너무 작으
면 속도가 느려집니다. 배치 크기는 한 번에 처리하는 데이터 개수를 결정하며, 크기에 따라 학습 안정성과 메모리 사용량이 달라집니다. 옵티마이저는 모델을 업데이트하는 방식과 관련된 것으
로, Adam, SGD 등 여러 종류가 있으며 학습 성능을 좌우합니다. SFTConfig 는 이 외에도 가중치 감쇠, 학습 스케줄링, 혼합 정밀도 학습 (fp16) 등의 학습에 영향을 주는 다양한 설정을 포함할 수 있습니다. LoRA 를 적용하면 모델 전체가 아닌 일부 가중치만 학습하므로 LoRA 와 연관된 설정들은
LoraConfig 에서 관리하지만, 로라 튜닝 여부와 별개의 학습 과정 전반의 설정은 여전히 SFTConfig에서 관리됩니다. 따라서, SFTConfig 는 LoRAConfig 와 함께 사용되며 학습을 조정하는 역할을 합니다.
- SFTTrainer: 실제 학습을 수행하는 클래스입니다. 주어진 데이터셋을 이용해 파인 튜닝 과정을 자동으로 수행하며, 특정 부분만 업데이트하는 LoRA 같은 학습 기법도 적용 가능합니다. 모델, 데이터셋, 학습 설정을 한 번에 입력하여 효율적인 학습을 진행할 수 있도록 돕습니다.
인터넷을 통해 이번 실습에서 사용할 데이터를 다운로드하고 특정 형식으로 전처리를 진행해보겠습니다. 보다 쉽게 코드를 이해하기 위해서 주석에 번호를 달아 1 번부터 9 번까지 한글 주석으로 각 코드 블록이 어떤 역할을 하는지 구체적으로 명시하였습니다.

In [ ]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

허깅페이스 허브에서 앞서 설명한 RAG 데이터셋을 불러옵니다. load_dataset() 함수를 사용하는데, "iamjoon/klue-mrc-ko-rag-dataset"이라는 데이터셋을 불러옵니다. 이 데이터셋은 우리가 사용할 전체 데이터를 담고 있습니다.

In [ ]:
# 2. system_message 정의
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다."라고 답변하시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색결과:
--------------
{search_result}
"""

우리가 학습을 위해 사용할 시스템 프롬프트를 정의합니다. 이 프롬프트는 RAG 성능을 높이기 위해 여러 가지 중요한 지침을 담고 있는데, 첫째로 검색 결과를 바탕으로 답변을 생성하도록 합니다. 둘째로 검색 결과에 없는 내용으로 답변하지 말라는 제약을 둡니다. 셋째로 특정 질문에 대한 내용이 검색 결과에
없을 경우 그 사실을 명시적으로 알리도록 합니다. 넷째로 답변 시 참고한 문서는 반드시 [[ref1]] 과 같은
형식으로 표시하도록 요구합니다. 여러 문서를 참고한 경우 [[ref1]], [[ref5]] 와 같이 모든 참고 문서를 표
시하도록 합니다. 마지막으로 가능한 한 많은 문서를 참고하여 답변하도록 지시합니다. 프롬프트 끝의
{search_result}는 실제 검색 결과로 대체될 예정입니다. 학습을 할 때에도 우리가 원하는 방향으
로 거대 언어 모델이 답변하도록 상세한 시스템 프롬프트를 작성해야 합니다.

In [ ]:
# 3. 원본 데이터의 type별 분포 출력
print("원본 데이터의 type 분포: ")
for type_name in set(dataset['type']):
  print(f"{type_name}: {dataset['type'].count(type_name)}")

원본 데이터의 type별 분포를 확인합니다. set() 함수로 중복 없는 type 목록을 만들고, count() 메소드로 각 type이 몇번 등장하는지 계산해 출력합니다.


In [ ]:
# 4. train/test 분할 비율 설정
test_ratio = 0.8

train_data = []
test_data = []

train/test 데이터 분할 비율을 설정합니다. test_ratio 변수에 0.8 을 할당하여 전체 데이터의 80% 를
테스트 데이터로, 나머지 20% 를 학습 데이터로 사용하도록 지정합니다. 일반적으로는 학습 데이터의 양이 더 많고, 성능을 평가하기 위한 테스트 데이터의 양이 더 적게 합니다. 하지만 현 실습에서는 유료 클라우드인 런팟을 사용하므로 실습 시 과도한 학습 비용을 방지하기 위해서 학습 데이터를 적게 설정하였습니다. 그리고 분할된 데이터의
인덱스를 저장할 train_data와 test_data라는 빈 리스트를 생성합니다.


In [ ]:
# 5. type별로 순회하면서 train/test 데이터 분할
for type_name in set(dataset['type']):
  # 현재 type에 해당하는 데이터의 인덱스만 추출
  curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type']==type_name]

  # test_ratio에 따라 test 데이터 개수 계산
  test_size = int(len(curr_type_data) * test_ratio)

  # 현재 type의 데이터를 test_ratio 비율로 분할하여 추가
  test_data.extend(curr_type_data[:test_size])
  train_data.extend(curr_type_data[test_size:])

코드는 데이터셋의 균형을 유지하면서 학습용과 테스트용 데이터를 분리하는 부분입니다. 각 타입별로 동일한 비율로 분할하는 이유는, 만약 무작위로 전체 데이터를 나누면 특정 타입의 데이터가 한쪽으로 쏠릴 수 있기 때문입니다. 예를 들어 'synthetic_question' 타입이 학습 데이터에 너무 많이 들어가고 테스트 데이터에는 거의 없다면, 모델이 이 타입의 질문에 대해서만 잘 학습하게 될 위험이 있습니다. 이를 방지하기 위해 각 타입별로 80:20 의 비율을 유지하면서 분할합니다. 구현을 살펴보면, set(dataset['type'])으로 얻은 각 type에 대해 반복문을 실행합니다. 각 반복에서는 현재 type에 해당하는 모든 데이터의 인덱스를 리스트 컴프리헨션으로 추출합니다. 코드 [i for i in range(len(dataset))if dataset[i]['type'] == type_name]은 데이터셋을 순회하면서 현재 type과 일치하는 데이터의 인덱스만 수집합니다. 그 다음 test_ratio를 사용해 테스트 데이터 개수를 계산하고, 슬라이싱으로 현재 type의 데이터를 test_data와 train_data에 분배합니다. 이렇게 나눈 데이터를 각각 test_data와 train_data 리스트에 추가하는데, 이는 나중에 실제 학습과 테스트에 사용될 데이터의 인덱스를 보관하는 것입니다.

In [ ]:
# 6. OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
  # 검색 결과를 문서1, 문서2... 형태로 포매팅
  search_result = "\n-----\n".join([f"문서{idx + 1}: {result}"for idx, result in enumerate(sample["search_result"])])

  # OpenAI format으로 변환
  return{
      "messages": [
          {
              "role": "system",
              "content": system_message.format(search_result=search_result)
          },
          {
              "role": "user",
              "content": sample["question"],
          },
          {
              "role": "assistant",
              "content": sample["answer"]
          }
      ]
  }


OpenAI 형식으로 데이터를 변환하는 format_data() 함수를 정의합니다. OpenAI 형식이란 ChatGPT를 개발한 OpenAI 에서 만든 데이터의 표준 형식을 말하는데, messages라는 리스트 안에 각각의 대화를 역할 (role) 과 내용 (content) 으로 구분하여 담는 구조입니다. 이는 앞에서 GPT‑4 API 실습에서 보았던 형식을 의미합니다. system은 AI 의 행동 지침을, user는 사용자의 질문을, assistant는 AI 의 답변을 나타냅니다.

In [ ]:
# 7. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

앞서 분할한 train_data와 test_data의 각 샘플에 format_data()함수를 적용하여 최종 데이터셋을 생성합니다. 각 인덱스에 해당하는 데이터를 format_data()함수를 이용해 OpenAI형식으로 변환하여 train_dataset과 test_datset에 저장합니다.

In [ ]:
# 8. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

In [ ]:
# 9. 분할된 데이터의 type별 분포 출력
print("\n학습 데이터의 type 분포")
for type_name in set(dataset['type']):
  count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
  print(f"{type_name}:{count}")

print("\n테스트 데이터의 type 분포")
for type_name in set(dataset['type']):
  count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
  print(f"{type_name}:{count}")

분할된 데이터의 type별 분포를 출력합니다. 학습 데이터와 테스트 데이터 각각에 대해 type별 개수를 계산하고 출력합니다. 이는 데이터 분할이 각 type에 대해 균형있게 이루어졌는지 검증하는 데 사용됩니다.

이제 OpenAI 형식으로 전처리가 완료된 데이터를 하나 임의로 출력하여 형식을 이해해보겠습니다. 여기서는 학습 데이터 중 임의로 345 번 샘플을 출력해보겠습니다. messages 라는 리스트 안에 데이터가 저장되어져 있으므로 이를 출력합니다.

In [ ]:
train_dataset[345]["messages"]

role 이 system 인 경우, content 에는 위에서 작성한 시스템 프롬프트와 현재 샘플의 검색 결과가 저장되
어져 있습니다. 내용이 길어서 중략했습니다. role 이 user 인 경우, content 에는 현 샘플에서의 사용자의 질문이 저장되어져 있습니다. role 이 assistant 인 경우, content 에는 검색 결과와 사용자의 질문을 바탕
으로 거대 언어 모델이 답변해야 할 내용이 작성되어져 있습니다. 위와 같이 system, user, assistant 와 같
은 role 과 각각 해당하는 content 로 구성된 형태를 OpenAI 형식이라고 합니다. 이 형식은 학습을 하기 위
한 최종 형식이 아니며 전처리를 위한 중간 단계입니다. 다만, 전처리를 위한 중간 단계의 형태로 OpenAI
에서 제안한 형식을 주로 사용하는 것입니다. 학습을 위한 최종 형식은 뒤의 내용에서 거대 언어 모델의
토크나이저를 통해 한 번 더 전처리를 진행하고 나서 결정됩니다.


현재 train_dataset 과 test_dataset 은 데이터의 타입이 리스트입니다. 원활한 학습을 위해서는 타입을 Dataset 으로 변경해야 합니다.

In [ ]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))


타입이 변경된 것을 확인하였습니다. 이제 뒤에서 파인튜닝한 모델을 평가하기 위해 테스트 데이터를 저장합니다.

### 2. 템플릿 적용하기
이제 학습을 진행할 모델과 토크나이저를 로드합니다. 거대 언어 모델을 학습시 에는 반드시 로드해야할 것이 크게 두 가지 있습니다. 바로 학습할 모델과 이모델에 입력할 데이터를 전처리하기 위한 도구인 토크나이저입니다. 거대 언어 모델은 고유한 토크나이저를 갖고 있으므로 반드시 학습할 모델의 토크나이저를 로드해야 합니다.

모델을 로드할 때에는 AutoModelForCausalLM.from_pretrained() 안에 모델의 이름을 기재하고, 토크나이저를 로드할 때에는 AutoTokenizer.from_pretrained() 안에 모델의 이름을 기재합니다. 앞서 언급하였듯이 각 거대 언어 모델은 고유한 토크나이저를 갖고 있으므로 이 두 개의 코드 안에 들어가는 모델의 이름은 일반적으로 동일합니다.

In [ ]:
# 허깅 페이스 모델 이름
model_id = "Qwen/Qwen2-7B-Instruct"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id,
  device_map="auto",
  torch_dtype= torch.bfloat16,
                                             )
tokenizer = AutoTokenizer.from_pretrained(model_id)

여기서는 “ Qwen/Qwen2-7B-Instruct”라는 모델을 사용할 것입니다. 따라서 모델과 토크나이저 모두 해당 모델의 이름을 인자로 사용한 것을 알 수 있습니다. 해당 모델은 중국의 IT 회사가 공개한 모델
인 Qwen의 2 버전 모델로 한글에서도 뛰어난 성능을 갖고 있습니다. 한글 성능이 뛰어난 또 다른 모델들로는 미국 기업 Meta가 공개한 LLaMA 3.1버전(비록 한글이 공식 지원하는 언어가 아님에도),
Google 의 공개한 Gemma 3버전 등이 존재합니다.

토크나이저를 로드하였다면 이제 Qwen을 위한 챗템플릿(Chat Template)을 적용해야 합니다. 챗
템플릿이란 거대 언어 모델이 학습할 때 사용하는 특정한 대화 형식. 다시 말해 학습 데이터의 특정 형식
을 의미합니다. 우리는 사용하는 거대 언어 모델들은 이미 학습된 모델이고, 우리는 이를 로드하여 우리
의 데이터로 추가적으로 파인 튜닝하고는 합니다. 이러한 거대 언어 모델은 만들어졌을 당시에 특정 형식
에 맞추어서 학습이 된 상태이므로 파인 튜닝 시에도 이 형식을 지켜주어야 합니다.


즉, 거대 언어 모델이 만들어질 당시, 첫 학습 때 사용된 템플릿과 다른 형식으로 데이터를 가공하여 파인
튜닝을 할 경우에는 모델이 제대로 된 성능을 내지 못하여 문맥을 제대로 인식하지 못하거나, 예상한 답변
을 제대로 생성하지 못할 가능성이 높아집니다.

이러한 챗 템플릿은 거대 언어 모델에 따라서 사용하는 챗 템플릿이 다를 수 있습니다. 예를 들어, 우리가
실습에 사용할 거대 언어 모델인 Qwen은 다음과 같은 챗 템플릿에 따라서 이미 학습된 모델입니다. 따라
서 파인 튜닝 할 데이터도 아래의 챗 템플릿 형식에 맞추어서 데이터를 가공하고 학습해야만 합니다.


In [ ]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

Qwen 의 챗 템플릿에 맞추어서 데이터가 가공되었습니다. <|im_start|>system와 <|im_end|>사이에는 시스템 프롬프트가 입력됩니다. 현재 실습하는 데이터 기준으로는 지시사항과 사용자의 질
문에 대한 검색 결과가 이에 해당됩니다. <|im_start|>user와 <|im_end|> 사이에는 사용자의 질문이 들어가며, <|im_start|>
assistant과 <|im_end|> 사이에는 실제 챗봇의 답변이 들어갑니다.

###3. 로라 학습을 위한 설정값
이제 학습을 위해 각종 설정값들을 지정해야 합니다. 이번 실습에서는 LoRA(Low‑Rank Adaptation) 튜닝 방식을 사용하여 파인 튜닝을 해볼 것입니다. 로라 튜닝는 거대한 언어 모델을 학습할 때 풀 파인튜닝(Full Finetuning) 보다 연산량과 GPU 메모리 사용량을 줄여 학습할 수 있도록 도와주는 방법입니다.

일반적으로 거대 언어 모델을 풀 파인 튜닝하려면 모델 내의 모든 변수(일반적으로 모델 내에 존재하며 학습 시에 값이 업데이트 되는 변수들을 “가중치” 라고 부릅니다.) 를 업데이트해야 하지만, 이렇게 하면 학습에 필요한 연산량이 급격히 증가하고, 많은 GPU 메모리를 소비하게 됩니다. 따라서 개인 연구자나
제한된 자원을 가진 환경에서는 풀 파인튜닝이 부담스러울 수 있습니다.

로라는 이러한 문제를 해결하기 위해 기존 모델의 가중치는 그대로 두고, 추가로 작은 변수를 가진 행렬을 덧붙여 이 추가적인 가중치 행렬만 학습하는 방식을 사용합니다. 즉, 모델 전체를 수정하는 풀 파인 튜닝 방식과는 달리, 필요한 가중치만 학습하는 방식입니다. 이 방식의 장점은 다음과 같습니다.

이 방식의 장점은 다음과 같습니다.
- 빠른 학습 속도 → 학습해야 하는 변수의 수가 줄어들어 학습 속도가 빨라집니다.
- 하드웨어 요구량 감소 → 기존 모델을 학습하는 것보다 훨씬 적은 연산량으로도 조정이 가능하여,
많은 양의 GPU 리소스가 없더라도 학습할 수 있습니다.

다음은 로라 튜닝을 위해서 각종 설정값들을 설정하는 코드입니다.

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

- lora_alpha: 로라 튜닝 후 기존 거대 언어 모델의 예측 결과에 얼마나 영향을 줄지를 조정하는 값입니다. lora_alpha 값이 크면 학습한 정보가 더 강하게 반영됩니다. lora_alpha 값이 작으면 기존 모델의 원래 특성이 더 많이 유지됩니다. 예를 들어, lora_alpha=1이면 기존 모델이
거의 그대로 유지되며, 로라 튜닝 후의 효과가 미미합니다. lora_alpha=100이면 기존 모델보다 로라 튜닝의 정보가 더 강하게 적용됩니다. 즉, lora_alpha는 로라 튜닝 후 기존 모델의 출력을 얼마나 변경할지를 결정하는 값입니다.
- lora_dropout: 학습할 때 특정 정보를 일부러 사용하지 않도록 설정하여 모델이 특정 데이터에만 의존하지 않도록 만드는 값. 모델이 학습 데이터에서 관측한 특정한 패턴에만 너무 의존하면, 새로운 데이터를 만났을 때 성능이 떨어질 수 있습니다. 이를 방지하기 위해 일부 정보를 무작위로 사용하지 않도록 설정하면서 학습하는 방법이 사용됩니다. lora_dropout은 학습할 때 일부 정보를 제외하여, 모델이 특정 데이터에 과도하게 의존하지 않도록 조정하는 역할을 합니다. 예를 들어
lora_dropout=0.1로 설정하면, 학습 과정에서 일부 정보 (약 10%) 가 의도적으로 제외된 상태로 학습이 진행됩니다. 이렇게 하면 특정 데이터에만 최적화되지 않고, 다양한 상황에서도 잘 작
동할 수 있도록 학습할 수 있습니다. 즉, lora_dropout은 모델이 새로운 데이터에도 적응할 수 있도록 돕는 역할을 합니다.
- r: 로라 튜닝이 학습하는 정보의 양을 결정하는 값. 로라 튜닝은 기존 모델 전체를 수정하는 것이 아니라, 특정한 부분만 선택적으로 학습합니다. 이때 학습할 정보의 크기를 결정하는 값이 r입니다. r값이 크면 더 많은 정보를 학습할 수 있지만, 그만큼 메모리 사용량과 연산량이 증가합니다. r 값이 작으면 메모리는 절약되지만, 학습할 수 있는 정보의 범위도 줄어듭니다. 예를 들어, r=2이면 로라 튜닝이 모델을 매우 작은 범위에서만 조정합니다. r=64이면 모델의 특정 부분을 더 넓은 범위에서 조정할 수 있습니다. 즉, r은 LoRA 가 학습할 수 있는 정보의 크기를 조정하는 역할을 합니다.
- bias: 로라 튜닝이 이루어지는 과정에서 거대 언어 모델의 편향값을 조정할지를 결정하는 값. 거대 언어 모델은 입력을 처리하는 과정에서 출력값을 조정하는 요소 중 하나로 편향값 (Bias) 이라는
것을 포함하고 있습니다. LoRA 는 기본적으로 이러한 편향값을 변경하지 않지만, 필요에 따라 로라 튜닝이 편향값까지 학습하도록 설정할 수도 있습니다. 여기서 선택한 "none" 값은 기존 모델의 편향값을 조정하지 않는다는 설정을 가집니다. 만약, "all" 값을 사용할 경우에는 기존 모델의 편향값까지 로라 튜닝이 함께 학습하여 더 큰 변화를 줄 수 있습니다.
- target_modules: 로라 튜닝을 적용할 특정 부분을 선택하는 값. 모델은 여러 개의 단계 (구성요소) 로 이루어져 있으며, 모든 부분을 로라 튜닝 방식으로 학습하는 것이 아니라, 필요한 부분만
선택적으로 학습할 수도 있습니다. "q_proj"와 "v_proj"는 모델이 입력을 처리하는 과정에서 중요한 역할을 하는 부분입니다. 이 부분을 LoRA 방식으로 학습하면 모델이 기존보다 더 유연하
게 작동할 수 있도록 조정할 수 있습니다. 즉, target_modules는 어떤 부분을 로라 튜닝으로 학습할지를 정하는 값입니다
- task_type: 로라 튜닝이 적용될 모델의 작업 유형을 지정하는 값. task_type 은 로라 튜닝을 적용할 모델이 어떤 방식으로 동작하는지를 설정하는 값입니다. 모델이 수행하는 작업 방식에 따라
로라 튜닝 방식도 달라지므로, 정확하게 설정해야 합니다. "CAUSAL_LM"은 입력된 텍스트를 기반으로 다음 단어를 예측하는 방식으로 학습하는 모델에 사용됩니다. 이는 거대 언어 모델 (GPT‑4,
Qwen, LLaMA 등) 과 같이 왼쪽에서 오른쪽으로 순차적으로 문장을 생성하는 모델에서 필요합니다.

###4. 학습을 위한 설정값
이제 모델을 학습할 때 필요한 다양한 설정값을 정의하는 도구인 SFTConfig 를 설정합니다. 학습 과정에서 모델이 어떻게 업데이트될지를 조정하는 값입니다. LoRA와 연관된 설정들은 LoraConfig 에서 관리하지만, 로라 튜닝 여부와 별개의 학습 과정 전반의 설정은 여전히 SFTConfig에서 관리됩니다. 따라서, SFTConfig는 LoRAConfig와 함께 사용되며 학습을 조정하는 역할을 합니다. 다음은 학습을 위해서 각종 설정값들을 설정하는 코드입니다.

In [ ]:
args = SFTConfig(
output_dir="qwen2-7b-rag-ko",           # 저장될 디렉토리와 저장소ID
num_train_epochs=3,                     # 학습할 총 에포크 수
per_device_train_batch_size=2,          # GPU당 배치 크기
gradient_accumulation_steps=2,          # 그래디언트 누적 스텝 수
gradient_checkpointing=True,            # 메모리 절약을 위한 체크 포인팅
optim="adamw_torch_fused",              # 최적화기
logging_steps=10,                       # 로그 기록 주기
save_strategy="steps",                  # 저장 전략
save_steps=50,                          # 저장 주기
bf16=True,                              # bfloat16 사용
learning_rate=1e-4,                     # 학습률
max_grad_norm=0.3,                      # 그래디언트 클리핑
warmup_ratio=0.03,                      # 워밍업 비율
lr_scheduler_type="constant",           # 고정 학습률
push_to_hub=False,                      # 허브 업로드 안함
remove_unused_columns=False,
dataset_kwargs={"skip_prepare_dataset": True},
report_to=None
)


- output_dir: 학습된 모델이 저장될 위치. 모델 학습이 끝난 후, 결과가 저장될 폴더를 지정하는 값입니다. 정확히는 로라 어댑터 (LoRA Adapter) 가 저장될 위치입니다. 로라 어댑터란 트랜스포
머 모델에 로라 튜닝을 적용할 때 새롭게 추가되는 두 개의 가중치 행렬 𝐴 와 𝐵 (그리고 경우에 따라 스케일 파라미터 𝛼 까지 포함) 를 통칭해서 부르는 말입니다. 학습하는 동안에 학습을 얼마나 했
는지에 따라서 로라 어댑터가 계속해서 저장될 것입니다.
- "qwen2-7b-rag-ko"로 설정되어 있으므로, 학습이 완료된 모델과 관련된 파일이 이 폴더에 저장됩니다.
- num_train_epochs: 학습할 총 횟수. num_train_epochs=3이면, 학습 데이터를 3 번 반복해서 학습합니다. 모델이 데이터를 여러 번 학습할수록 더 많이 배울 수 있지만, 너무 많이 반복하면 기존 학습 데이터에 너무 맞춰져 새로운 데이터에서 성능이 떨어질 수 있습니다.
- per_device_train_batch_size: 한번학습할때처리하는데이터개수. per_device_train_batch_=2이면, 한 번에 2 개의 데이터를 사용해서 학습합니다. 학습을 할 때 데이터를 하나씩 처리하는 것
이 아니라 여러 개를 묶어서 한 번에 학습하는데, 이를 “배치 (batch)” 라고 합니다. 배치 크기가 크면 학습 속도가 빨라질 수 있지만, 메모리를 많이 사용합니다.
- gradient_accumulation_steps: 여러 번 계산한 결과를 모아서 한 번에 적용하는 기능. gradient_accumulation_steps=2이면, 2 번 계산한 결과를 모아서 한 번에 모델을 업데이트합니다. 메모리가 부족할 때 작은 배치를 여러 번 모아서 학습할 수 있도록 도와주는 기능입니다.
- gradient_checkpointing: 메모리를 절약하는 기능. gradient_checkpointing=True이면, 학습할 때 일부 정보를 저장하지 않고, 필요할 때 다시 계산하여 메모리를 절약하는 기
능이 활성화됩니다. 속도가 조금 느려질 수 있지만, 메모리를 적게 쓰는 장점이 있습니다.
- optim: 모델을 업데이트할 때 사용하는 최적화 방법. "adamw_torch_fused"는 AdamW 최적화 기법을 사용하여 모델을 업데이트하는 방식입니다. AdamW는 안정적인 학습이 가능하도록 제
안된 학습 방식입니다.
- logging_steps: 학습 중 진행 상황을 얼마나 자주 기록할지 결정하는 값. logging_steps=10이면, 10 번의 학습 스텝마다 진행 상태를 기록합니다. 너무 자주 기록하면 속도가 느려질 수 있
고, 너무 드물면 학습 상태를 파악하기 어려울 수 있습니다.
- save_strategy 및 save_steps: 모델을 저장하는 방식과 주기. save_strategy="
steps"이면, 지정된 스텝 (step) 마다 모델을 저장합니다. save_steps=50이면, 50 번의 학습 스텝마다 모델이 저장됩니다. 모델이 중간에 저장되지 않으면 학습 중 문제가 발생할 경우 처음부터 다시 학습해야 할 수도 있기 때문에, 적절한 주기로 저장하는 것이 중요합니다.
- bf16: bfloat16(16 비트 부동소수점) 사용 여부. bf16=True이면, bfloat16 형식을 사용하여 모델을 학습합니다. bfloat16 은 메모리를 절약하면서도 연산의 정확도를 유지할 수 있는 데이터 형
식으로, NVIDIA A100 과 같은 최신 GPU 에서 성능을 최적화할 수 있습니다.
- learning_rate: 학습 속도를 조절하는 값. learning_rate=1e-4이면, 학습 속도를
0.0001(1 × 10⁻⁴) 로 설정합니다. 값이 너무 크면 모델이 불안정하게 학습될 수 있고, 값이 너무 작으면 학습이 느려질 수 있습니다.
- max_grad_norm: 그래디언트 클리핑 설정. max_grad_norm=0.3이면, 모델이 한 번 업데이트될 때 변화량을 제한하여 너무 급격한 변화가 일어나지 않도록 조정합니다. 변화량이 너무 커지
는 경우 모델이 불안정해질 수 있기 때문에 이를 제한하는 기능입니다.
- warmup_ratio: 학습 초반에 천천히 시작하도록 조정하는 값. warmup_ratio=0.03이면, 초반 3% 구간 동안 학습률을 서서히 증가시키며 적응하는 방식으로 학습을 진행합니다. 처음부터 너
무 빠르게 학습하면 모델이 불안정할 수 있기 때문에, 일정 구간 동안 천천히 학습 속도를 올려 안정적으로 학습할 수 있도록 합니다.
- lr_scheduler_type: 학습률을 어떻게 조정할지 결정하는 방법. lr_scheduler_type="constant"이면, 학습률을 일정하게 유지하는 방식으로 학습합니다. 일반적으로 학습이 진행됨
에 따라 학습 속도를 줄이는 방식도 있지만, 여기서는 처음부터 끝까지 같은 학습 속도를 유지하는 방식을 선택했습니다.
- push_to_hub: 학습된 모델을 허깅페이스 웹 사이트를 통해 접근 가능한 모델 저장소에 업로드 할지 여부. push_to_hub=False이면, 학습된 모델을 업로드하지 않습니다. 허깅페이스 모델
저장소에 자동으로 업로드하려면 True로 설정해야 합니다.
- remove_unused_columns: 불 필 요 한 컬 럼 제 거 여 부. remove_unused_columns=
False이면, 데이터에서 사용하지 않는 컬럼을 자동으로 제거하지 않습니다. 이 값을 True 로 설정하면, 모델이 사용하지 않는 데이터를 자동으로 정리하여 학습을 최적화할 수 있습니다.
- dataset_kwargs: 데이터셋관련추가설정. dataset_kwargs={"skip_prepare_dataset
": True}이면, 데이터를 미리 준비하는 과정을 생략합니다. 데이터셋이 이미 준비되어 있는 경우, 불필요한 처리를 생략하여 속도를 높일 수 있습니다.
- report_to: 학습 과정의 정보를 어디로 보고할지 지정하는 값. report_to=None이면, 학습
과정을 별도로 로그를 저장하는 툴에 기록하지 않습니다.
SFTConfig 는 로라 튜닝을 최적화하기 위한 다양한 설정을 포함하고 있으며, 학습 방식, 저장 방법, 최적화 방식 등을 조정하는 역할을 합니다. 모델이 어떻게 학습하고, 얼마나 자주 저장되며, 어떤 방식으로 최적화되는지를 결정하는 중요한 요소입니다. 각각의 값을 조정함으로써, 하드웨어 성능과 원하는 학습 방식에 맞게 최적화할 수 있습니다.


### 5. 정수 인코딩
거대 언어 모델에 학습 데이터를 전달하기 전에 학습 데이터는 수치화 작업을 거칩니다. 더 정확히는 텍스트 데이터는 전부 거대 언어 모델이 이해할 수 있는 정수 데이터로 변환되어야 합니다. 이 과정을 일반적으로 인코딩 (encoding) 이라 부르며 앞서 로드한 토크나이저를 통해 가능합니다. 예를 통해 인코딩 과
정을 이해해보겠습니다. 거대 언어 모델을 학습할 때에는 입력과 출력 두 가지가 존재해야 합니다. 여기서는 입력과 출력의 각 변수명을 input_ids와 labels라고 해봅시다. 또한, 모델이 학습해야 하는 데
이터는 다음과 같다고 가정하겠습니다.

In [ ]:
"""
- 시스템 프롬프트: "당신은 친절한 AI어시던트입니다."
- 유저 프롬프트: "안녕하세요, 오늘 날씨는 어떤가요?"
- 거대 언어 모델의 응답: "안녕하세요! 오늘 날씨는 맑고 화창합니다."
"""

이 데이터를 학습하기 위해서는 앞서 진행된 바와 같이 챗 템플릿을 적용해야 합니다.

In [ ]:
"""
<|im_start|>system
당신은 친절한 AI어시던트입니다.<|im_end|>
<|im_start|>user
안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>
<|im_start|>assistant
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
"""

챗 템플릿이 적용되면 이로부터 inputs와 labels를 생성합니다. 각 토큰이 어떤정수로 변환 되어야 하는지에 대한 정보는 앞서 로드한 토크나이저를 통해 얻을 수 있으므로, 토크나이저를 이용하여 인코딩 작업을 통해 정수로 변환하면 input_ids는 다음과 같습니다.

In [ ]:
input_ids = [
    151644, 198, # <|im_start|>system (줄바꿈)
    8948, 198, 64795, 82528, 33704, 90711, 250, 126550, # 당신은 친절한 AI어시던트입니다.
    198, 151645, 198, # <|im_end|> (줄바꿈)
    151644, 198, # <|im_start|>user (줄 바 꿈)
    126246, 144370, 91145, 133857, 37195, 254, 135562, 16560, 129273, 19969,
    35711, 30, # 안녕하세요, 오늘 날씨는 어떤가요??
    198, 151645, 198, # <|im_end|> (줄 바 꿈)
    151644, 198, # <|im_start|>assistant (줄 바 꿈)
    77091, 198, 126246, 144370, 91145, 0, 133857, 37195, 254, 135562, 16560,32985, 239, 34395, 46832, 242, 130095, 60838, 13, # 안녕하세요! 오늘 날씨는 맑고 화창합니다.
    198, 151645, 198 # <|im_end|> (줄 바 꿈)
]


어떤 토큰이 어떤 정수로 맵핑되는지는 거대 언어 모델마다 전부 다를 수 있습니다. 예를 들어 우리가 사용하는 거대 언어 모델 Qwen 에서 <|im_start|>는 151644 라는 정수입니다. 하지만 다른 거대 언어모델에서는 151644 가 다른 의미를 가진 토큰일 수 있으므로 반드시 사용하고 있는 거대 언어 모델의 토크나이저를 사용해야 합니다. 이제 labels를 만드는 방법에 대해서 이해해봅시다.
위에 inputs_ids에서 거대 언어 모델의 답변에 해당하는 응답 부분을 보겠습니다. 거대 언어 모델의 응답에 해당하는 텍스트는 “안녕하세요! 오늘 날씨는 맑고 화창합니다.” 이며, 위의 예시에서 해당하는
정수는 [77091, 198, 126246, 144370, 91145, 0, 133857, 37195, 254, 135562, 16560, 32985, 239, 34395,
46832, 242, 130095, 60838, 13] 입니다.
input_ids에는 거대 언어 모델의 응답 뿐만 아니라, 시스템 프롬프트와 유저 프롬프트까지 모두 포함 되어 있지만, 실제로 모델이 학습해야 하는 것은 시스템 프롬프트와 유저 프롬프트 (사용자의 질문) 을 보고 적절한 응답을 생성하는 것이지, 시스템 프롬프트나 사용자 입력은 거대 언어 모델의 입력에 해당할뿐, 실제로 거대 언어 모델이 생성해야 하는 부분은 아닙니다. 따라서, 모델이 직접 생성할 필요가 없는 부분은 input_ids에서 ‑100 으로 처리하여 labels를 만듭니다. 실제 학습 시 labels에서 ‑100 인 토큰들은 생성을 위한 학습 대상에서 제외할 수 있습니다.


In [ ]:
labels = [
    -100, -100, # <|im_start|>system (줄바꿈)
    100, -100, -100, -100, -100, -100, -100, -100, # 당신은 친절한 AI어시던트입니다.
    100, -100, -100, # <|im_end|> (줄바꿈)
    -100, -100, # <|im_start|>user (줄바꿈)
    -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, # 안녕하세요, 오늘 날씨는 어떤가요?
    -100, -100, -100, # <|im_end|> (줄바꿈)
    -100, -100, # <|im_start|>assistant (줄바꿈)
    77091, 198, 126246, 144370, 91145, 0, 133857, 37195, 254, 135562, 16560,
    32985, 239, 34395, 46832, 242, 130095, 60838, 13, # 안녕하세요! 오늘 날씨는 맑고 화창합니다.
    -100, -100, -100 # <|im_end|> (줄바꿈)
]


이렇게 설정하면 모델은 labels에서 ‑100 이 아닌 부분만 학습하여 거대 언어 모델이 생성해야 하는 응답만 학습하도록 유도됩니다. 이 방식은 모델이 불필요한 부분을 학습하려고 하지 않고, 올바른 답변
을 생성하는 데 집중할 수 있도록 하는 기본적인 학습 구조입니다. 위와 같은 전처리를 진행하는 함수로 collate_fn을 작성해보겠습니다.

###6. 정수 인코딩 적용하기

In [ ]:
def collate_fn(batch):
  new_batch = {
    "input_ids": [],
    "attention_mask": [],
    "labels": []
  }

  for example in batch:
    # message의 각 내용에서 개행문자 제거
    clean_messages = []
    for message in example["messages"]:
      clean_message = {
          "role": message["role"],
          "content": message["content"]
      }
      clean_messages.append(clean_message)

    # 깨끗해진 메시지로 템플릿 적용
    text = tokenizer.apply_chat_template(
        clean_messages,
        tokenize=False,
        add_generation_prompt=False
    ).strip()

    # 텍스트를 토큰화
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=max_seq_length,
        padding=False,
        return_tensors=None,
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # 레이블 초기화
    labels = [-100] * len(input_ids)

    # assistant 응답 부분 찾기
    im_start = "<|im_start|>"
    im_end = "<|im_end|>"
    assistant = "assistant"

    # 토큰 ID가져오기
    im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
    im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
    assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

    i = 0
    while i < len(input_ids):
      # <|im_start|>assistant 찾기
      if (i+len(im_start_tokens) <= len(input_ids) and
          input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

        # assistant 토큰 찾기
        assistant_pos = i + len(im_start_tokens)
        if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
            input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

          # assistant 응답의 시작 위치로 이동
          current_pos = assistant_pos + len(assistant_tokens)

          # <|im_end|>를 찾을 때까지 레이블 설정
          while current_pos < len(input_ids):
            if input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens:
              # <|im_end|> 토큰도 레이블에 포함
              for j in range(len(im_end_tokens)):
                labels[current_pos + j] = input_ids[current_pos + j]
              break
            labels[current_pos] = input_ids[current_pos]
            current_pos += 1

          i = current_pos + len(im_end_tokens)

      i +=1

    new_batch["input_ids"].append(input_ids)
    new_batch["attention_mask"].append(attention_mask)
    new_batch["labels"].append(labels)

  # 패딩 적용
  max_length = max(len(ids) for ids in new_batch["input_ids"])

  for i in range(len(new_batch["input_ids"])):
    padding_length = max_length - len(new_batch["input_ids"][i])
    new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
    new_batch["attention_mask"][i].extend([0] * padding_length)
    new_batch["labels"][i].extend([-100] * padding_length)

  # 텐서로 변환
  for k, v in new_batch.items():
    new_batch[k] = torch.tensor(v)

  return new_batch

In [ ]:
# 최대 시퀀스 길이 정의
max_seq_length = 1024

In [ ]:
example = train_dataset[0]
batch = collate_fn([example])
print('입 력 에 대 한 정 수 인 코 딩 결 과:')
print(batch["input_ids"][0].tolist())
print('레 이 블 에 대 한 정 수 인 코 딩 결 과:')
print(batch["labels"][0].tolist())

실제로는 출력 결과가 길어서 지면의 한계로 중략하였습니다. 출력 결과를 살펴보면 입력에 대한 정수 인코딩 결과와 레이블에 대한 정수 인코딩 결과는 길이는 동일합니다. 하지만 레이블에 대한 정수 인코딩 결과에서는 거대 언어 모델의 실제 답변 부분을 제외하고는 전부 ‐100 으로 채워진 값이 출력됩니다.

###7. 학습하기
이제 전처리가 끝났습니다. 실제 학습을 진행하겠습니다.


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

# 학습 시작
trainer.train()  # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()  # 최종 모델을 저

### 8. 모델 테스트
테스트 데이터에 대해서 학습전, 후 모델의 결과를 비교해보겠습니다. 아래의 코드는 테스트 데이터에서 모델의 입려과 정답에 해당하는 레이블을 별도 분리하여 저장하는 코드입니다.

In [ ]:
prompt_lst = []
label_lst = []
for prompt in test_dataset["messages"]:
  text = tokenizer.apply_chat_template(
      prompt, tokenize=False, add_generation_prompt=False
      )
  input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
  label = text.split('<|im_start|>assistant')[1]
  prompt_lst.append(input)
  label_lst.append(label)

전처리가 끝났으니 임의로 42 번 샘플의 입력과 레이블을 출력해보겠습니다. 먼저 거대 언어 모델의 입력으로 사용할 42 번 샘플의 입력입니다.

In [ ]:
print(prompt_lst[42])

In [ ]:
print(label_lst[42])

테스트 데이터 전처리 결과를 확인했습니다. 이제 파인 튜닝 전, 후 모델을 호출하여 실제 출력 결과를 확인해보겠습니다.

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

base_model_id = "Qwen/Qwen2-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto",
                                             torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

def test_inference(pipe, prompt):
  outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
  return outputs[0]['generated_text'][len(prompt):].strip()

prompt=prompt_lst[42]
label=label_lst[42]
pred=test_inference(pipe, prompt)

print(f"모 델 의 예 측:\n{test_inference(pipe, prompt)}")
print(f"정 답:\n{label}")

base_model_id에는 Qwen2-7B-Instruct 이라는 파인 튜닝 전 기본 모델의 이름을 저장합니다.
모델을불러올때는AutoModelForCausalLM.from_pretrained()를사용합니다. pipeline
은 텍스트 생성을 위한 huggingface 의 유틸리티로, 앞서 로드한 모델과 토크나이저를 사용하여 빠르게 입력에 대한 출력 결과를 얻게 해주는 도구입니다. eos_token은 모델이 텍스트 생성을 멈추는 시점을 알려주는 특수 토큰, 구체적으로는 종료 토큰입니다. 해당 토큰을 예측하는 순간 더 이상 생성을 할 필요가 없으니 기계적으로 멈춰야 합니다. "<|im_end|>"를 토크나이저로 변환해서 그 ID 를 저장합니다. test_inference 함수는 실제로 모델이 텍스트를 생성하는 부분입니다. max_new_tokens=1024는 최대 1024 개의 새로운 토큰을 생성하도록 제한합니다. 다시 말해 답변은 해당 길이를 넘어서 생성될 수 없습니다. 그 후 prompt_lst의 42 번째 샘플을 입력으로 넣어 모델의 예측 결과를 pred에 저장하고, 실제 정답 결과와 비교합니다.

실행 결과를 보면 학습하지 않은 모델의 경우에는 정답과는 달리 [[ref 문서 번호]] 와 같은 출처를 넣지 않
을 뿐만 아니라 전반적으로 답변의 길이도 다소 짧은 듯한 모습을 보여줍니다. 이번에는 파인 튜닝 후의
모델을 호출하여 예측과 정답을 비교해봅시다.

In [ ]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id,
                              device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

prompt=prompt_lst[42]
label=label_lst[42]
pred=test_inference(pipe, prompt)

print(f"모델의 예측:\n{test_inference(pipe, prompt)}")
print(f"정답:\n{label}")

기본 모델 호출과 코드는 전반적으로 동일하지만 모델의 최종 로라 어댑터의 학습 결과가 저장된
"qwen2-7b-rag-ko/checkpoint-285"에서 로라 어댑터를 로드, 그리고 기존 LLM 과 병합 후 학
습된 모델을 기준으로 평가하는 코드입니다.

파인 튜닝 후의 모델은 기본 모델과는 달리 답변이 전반적으로 정돈되고, [[ref 문서 번호]] 를 지키라는 지시사항도 충실히 지키는 모습을 보여줍니다. 이처럼 파인 튜닝을 통해 RAG 에서 사용되는 거대 언어 모델
의 답변 성능을 높일 수 있습니다. 여러분들도 여러분들만의 도메인에 맞는 RAG 학습 데이터를 구축하여 RAG 의 성능을 높여보시기 바랍니다.